<a href="https://colab.research.google.com/github/pratim808/TTS_Bengali/blob/main/TTS_Bengali_Fine-Tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets soundfile accelerate speechbrain==0.5.16

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.6/630.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 34.3 MB/s eta 0:00:00


In [ ]:
'''from datasets import load_dataset, Audio
dataset = load_dataset("spygaurad/Bengali_Competition_Dataset", split="train")
dataset'''

'from datasets import load_dataset, Audio\ndataset = load_dataset("spygaurad/Bengali_Competition_Dataset", split="train")\ndataset'

In [ ]:
#dataset.save_to_disk("/content/drive/MyDrive/TTH_bengali/Dataset")

In [ ]:
from datasets import load_from_disk
#dataset.save_to_disk("/content/drive/MyDrive/TTH_bengali/Dataset")
dataset = load_from_disk("/content/drive/MyDrive/TTH_bengali/Dataset")

Loading dataset from disk:   0%|          | 0/34 [00:00<?, ?it/s]

In [ ]:
#dataset = load_from_disk("/content/drive/MyDrive/TTH_bengali/Dataset")

In [ ]:
dataset

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 629442
})

In [ ]:
dataset

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 629442
})

In [ ]:
len(dataset)

629442

In [ ]:
print(dataset)

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 629442
})


In [ ]:
# Calculate the number of rows for half the dataset
half_size = len(dataset) // 60

# Select the first half of the dataset
dataset = dataset.select(range(half_size))

print(dataset)

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 10490
})


In [ ]:
len(dataset) // 60

174

In [ ]:
from datasets import Audio
dataset= dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
from transformers import SpeechT5Processor

checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
tokenizer = processor.tokenizer

In [ ]:
dataset[2:5]

{'audio': [{'path': '9e92be09f554.mp3',
   'array': array([-1.16415322e-10, -2.32830644e-10,  1.74622983e-10, ...,
          -4.84548509e-05, -1.08109787e-04, -3.38783488e-04]),
   'sampling_rate': 16000},
  {'path': '45b4430ad5f3.mp3',
   'array': array([-7.27595761e-12, -1.72803993e-11, -4.00177669e-11, ...,
          -3.18717923e-08,  6.84741508e-09, -1.87487785e-08]),
   'sampling_rate': 16000},
  {'path': 'ff35789066a6.mp3',
   'array': array([ 3.41060513e-13,  7.95807864e-13, -3.97903932e-13, ...,
           3.72157519e-05,  3.30214389e-05,  2.77965446e-05]),
   'sampling_rate': 16000}],
 'sentence': ['অস্ট্রিয়ার সৈন্যরা যুদ্ধক্ষেত্র ত্যাগ করে পালিয়ে গেল।',
  'ফুটবল খেলা থেকে অবসর নেয়ার পর সমাজকর্মী ও ন্যায়বিচারের প্রচারণামূলক কর্মকাণ্ডে নিযুক্ত রয়েছেন রাই।',
  'সে তার ভালো যত্ন নেবে।']}

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/10490 [00:00<?, ? examples/s]

In [ ]:
dataset_vocab - tokenizer_vocab

{' ',
 '।',
 'ঁ',
 'ং',
 'ঃ',
 'অ',
 'আ',
 'ই',
 'ঈ',
 'উ',
 'ঊ',
 'ঋ',
 'এ',
 'ঐ',
 'ও',
 'ঔ',
 'ক',
 'খ',
 'গ',
 'ঘ',
 'ঙ',
 'চ',
 'ছ',
 'জ',
 'ঝ',
 'ঞ',
 'ট',
 'ঠ',
 'ড',
 'ঢ',
 'ণ',
 'ত',
 'থ',
 'দ',
 'ধ',
 'ন',
 'প',
 'ফ',
 'ব',
 'ভ',
 'ম',
 'য',
 'র',
 'ল',
 'শ',
 'ষ',
 'স',
 'হ',
 '়',
 'া',
 'ি',
 'ী',
 'ু',
 'ূ',
 'ৃ',
 'ে',
 'ৈ',
 'ো',
 'ৌ',
 '্',
 'ৎ',
 'ড়',
 'ঢ়',
 'য়',
 '–',
 '‘',
 '’',
 '…'}

In [ ]:
import re

def normalize_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove specific signs
    text = re.sub(r"[৷–‘’]", "", text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Define a function to add the normalized_text column
def add_normalized_text(example):
    example['normalized_text'] = normalize_text(example['sentence'])
    return example

# Apply the function to the dataset
dataset = dataset.map(add_normalized_text)

# Print the first few examples to verify
print(dataset[2:5])

{'audio': [{'path': '9e92be09f554.mp3', 'array': array([-1.16415322e-10, -2.32830644e-10,  1.74622983e-10, ...,
       -4.84548509e-05, -1.08109787e-04, -3.38783488e-04]), 'sampling_rate': 16000}, {'path': '45b4430ad5f3.mp3', 'array': array([-7.27595761e-12, -1.72803993e-11, -4.00177669e-11, ...,
       -3.18717923e-08,  6.84741508e-09, -1.87487785e-08]), 'sampling_rate': 16000}, {'path': 'ff35789066a6.mp3', 'array': array([ 3.41060513e-13,  7.95807864e-13, -3.97903932e-13, ...,
        3.72157519e-05,  3.30214389e-05,  2.77965446e-05]), 'sampling_rate': 16000}], 'sentence': ['অস্ট্রিয়ার সৈন্যরা যুদ্ধক্ষেত্র ত্যাগ করে পালিয়ে গেল।', 'ফুটবল খেলা থেকে অবসর নেয়ার পর সমাজকর্মী ও ন্যায়বিচারের প্রচারণামূলক কর্মকাণ্ডে নিযুক্ত রয়েছেন রাই।', 'সে তার ভালো যত্ন নেবে।'], 'normalized_text': ['অস্ট্রিয়ার সৈন্যরা যুদ্ধক্ষেত্র ত্যাগ করে পালিয়ে গেল।', 'ফুটবল খেলা থেকে অবসর নেয়ার পর সমাজকর্মী ও ন্যায়বিচারের প্রচারণামূলক কর্মকাণ্ডে নিযুক্ত রয়েছেন রাই।', 'সে তার ভালো যত্ন নেবে।']}


In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["normalized_text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/10490 [00:00<?, ? examples/s]

In [ ]:
dataset_vocab - tokenizer_vocab

{' ',
 '।',
 'ঁ',
 'ং',
 'ঃ',
 'অ',
 'আ',
 'ই',
 'ঈ',
 'উ',
 'ঊ',
 'ঋ',
 'এ',
 'ঐ',
 'ও',
 'ঔ',
 'ক',
 'খ',
 'গ',
 'ঘ',
 'ঙ',
 'চ',
 'ছ',
 'জ',
 'ঝ',
 'ঞ',
 'ট',
 'ঠ',
 'ড',
 'ঢ',
 'ণ',
 'ত',
 'থ',
 'দ',
 'ধ',
 'ন',
 'প',
 'ফ',
 'ব',
 'ভ',
 'ম',
 'য',
 'র',
 'ল',
 'শ',
 'ষ',
 'স',
 'হ',
 '়',
 'া',
 'ি',
 'ী',
 'ু',
 'ূ',
 'ৃ',
 'ে',
 'ৈ',
 'ো',
 'ৌ',
 '্',
 'ৎ',
 'ড়',
 'ঢ়',
 'য়',
 '…'}

In [ ]:
replacements = [


    ('া', 'aa'),  # Long a
    ('ৌ', 'ou'),  # Ch as in "chair"
    ('ি', 'ei'),  # Silent g or slight elongation of the preceding vowel
    ('ী', 'ei'),   # Dotless i
    ('ে', 'ea'),   # Long i
    ('ৈ', 'oi'),  # Similar to German ö
    ('ো', 'o'),  # Sh as in "shoe"
    ('ু', 'eu'),  # Similar to German ü
    ('ূ', 'eu'),
    ( 'ৃ','ri'),
    ('়','ro'),
    ('অ','A'),
    ('আ','A'),
    ('ই','I'),
    ('ঈ','I'),
    ('উ','U'),
    ('ঊ','U'),
    ('ঋ','R'),
    ('এ','E'),
    ('ঐ','E'),
    ('ও','O'),
    ('ঔ','O'),
    ('ক','k'),
    ('খ','kh'),
    ('গ','g'),
    ('ঘ','gh'),
    ('ঙ','ng'),
    ('চ','ch'),
    ('ছ','chh'),
    ('জ','j'),
    ('ঝ','j'),
    ('ঞ','nj'),
    ('ট','t'),
    ('ঠ','th'),
    ('ড','d'),
    ('ঢ','dh'),
    ('ণ','n'),
    ('ত','t'),
    ('থ','th'),
    ('দ','d'),
    ('ধ','dh'),
    ('ন','n'),
    ('প','p'),
    ('ফ','f'),
    ('ব','b'),
    ('ভ','v'),
    ('ম','m'),
    ('য','y'),
    ('র','r'),
    ('ল','l'),
    ('শ','sh'),
    ('ষ','s'),
    ('স','s'),
    ('হ','h'),
    ('ড়','r'),
    ('ঢ়','r'),
    ('য়','y'),
    ('ৎ','t'),
    ('ং','ng'),
    ('ঃ','h'),
    ('ঁ','n'),



]

def cleanup_text(inputs):
    for src, dst in replacements:
        inputs["sentence"] = inputs["sentence"].replace(src, dst)
    return inputs

dataset = dataset.map(cleanup_text)

In [ ]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:147: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(path, map_location=device), strict=Fal

In [ ]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        text=example["normalized_text"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [ ]:
processed_example = prepare_dataset(dataset[0])
list(processed_example.keys())

['input_ids', 'labels', 'speaker_embeddings']

In [ ]:
processed_example["speaker_embeddings"].shape

(512,)

In [ ]:
#dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

In [ ]:
'''from datasets import load_from_disk
dataset.save_to_disk("/content/drive/MyDrive/TTH_bengali/Dataset_map")'''


'from datasets import load_from_disk\ndataset.save_to_disk("/content/drive/MyDrive/TTH_bengali/Dataset_map")'

In [ ]:
dataset_map = load_from_disk("/content/drive/MyDrive/TTH_bengali/Dataset_map")

In [ ]:
dataset_map

Dataset({
    features: ['input_ids', 'labels', 'speaker_embeddings'],
    num_rows: 10490
})

In [ ]:
dataset_map [2:5]

{'input_ids': [[4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 2],
  [4,
   3,
   4,
   3,
   4,
   3,
   4,
   3,
   4,
   3,
   4,
   3,
   4,
   3,
   4,
   3,
   4,
   3,
   4,
   3,
   4,
   3,
   4,
   3,
   4,
   3,
   4,
   3,
   2],
  [4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 2]],
 'labels': [[[-7.245288848876953,
    -7.032399654388428,
    -7.1932244300842285,
    -7.547857284545898,
    -7.318709850311279,
    -6.846534252166748,
    -6.8270158767700195,
    -7.112674236297607,
    -6.752033710479736,
    -6.394346237182617,
    -6.628307342529297,
    -6.8834991455078125,
    -7.0910468101501465,
    -6.8874006271362305,
    -6.992725849151611,
    -6.626160621643066,
    -6.861876010894775,
    -6.983675479888916,
    -6.682291030883789,
    -6.586693286895752,
    -6.611116886138916,
    -6.729509353637695,
    -6.629602432250977,
    -6.454680442810059,
    -6.3111138343811035,
    -6.569849967956543,
    -6.783853530883789,
    -6.902666091918945,
    -7.268970489501953,
    -6.992025

In [ ]:
len(dataset_map )

10490

In [ ]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200

dataset_map  = dataset_map .filter(is_not_too_long, input_columns=["input_ids"])
len(dataset)

10490

In [ ]:
dataset_map  = dataset_map.train_test_split(test_size=0.1)

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [ ]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

In [ ]:
from transformers import SpeechT5ForTextToSpeech

model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

In [ ]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(model.generate, use_cache=True)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/TTH_bengali/speecht5_finetuned_Bengali_shabdobhedii",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    warmup_steps=100,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset_map["train"],
    eval_dataset=dataset_map["test"],
    data_collator=data_collator,
    tokenizer=processor,
)

NameError: name 'training_args' is not defined

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
100,0.801800,0.720225
200,0.729500,0.693406
300,0.698100,0.662527
400,0.684500,0.652891
500,0.670800,0.636241
600,0.680800,0.635045
700,0.666600,0.630810
800,0.663300,0.634639
900,0.651800,0.631854
1000,0.651700,0.620663


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1876}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/e

Step,Training Loss,Validation Loss
100,0.801800,0.720225
200,0.729500,0.693406
300,0.698100,0.662527
400,0.684500,0.652891
500,0.670800,0.636241
600,0.680800,0.635045
700,0.666600,0.630810
800,0.663300,0.634639
900,0.651800,0.631854
1000,0.651700,0.620663


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1876}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/e

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
model = SpeechT5ForTextToSpeech.from_pretrained(
    "/content/drive/MyDrive/TTH_bengali/speecht5_finetuned_Bengali_shabdobhedii"
)

In [ ]:
example = dataset_map["test"][304]
speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)

In [ ]:
text = "কেমন আছো, আমি তোমাকে ভালবাসি!"

In [ ]:
import re

number_words = {
    0: "শূন্য", 1: "এক", 2: "দুই", 3: "তিন", 4: "চার", 5: "পাঁচ", 6: "ছয়", 7: "সাত", 8: "আট", 9: "নয়",
    10: "দশ", 11: "এগারো", 12: "বারো", 13: "তেরো", 14: "চৌদ্দ", 15: "পনেরো", 16: "ষোল", 17: "সতেরো",
    18: "আঠারো", 19: "ঊনিশ", 20: "বিশ", 30: "ত্রিশ", 40: "চল্লিশ", 50: "পঞ্চাশ", 60: "ষাট", 70: "সত্তর",
    80: "আশি", 90: "নব্বই", 100: "একশ", 1000: "এক হাজার"
}

def number_to_words(number):
    if number < 20:
        return number_words[number]
    elif number < 100:
        tens, unit = divmod(number, 10)
        return number_words[tens * 10] + (" " + number_words[unit] if unit else "")
    elif number < 1000:
        hundreds, remainder = divmod(number, 100)
        return (number_words[hundreds] + " শত" if hundreds > 1 else "একশ") + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000:
        thousands, remainder = divmod(number, 1000)
        return (number_to_words(thousands) + " হাজার" if thousands > 1 else "এক হাজার") + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000:
        millions, remainder = divmod(number, 1000000)
        return number_to_words(millions) + " মিলিয়ন" + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000000:
        billions, remainder = divmod(number, 1000000000)
        return number_to_words(billions) + " বিলিয়ন" + (" " + number_to_words(remainder) if remainder else "")
    else:
        return str(number)

def replace_numbers_with_words(text):

    def replace(match):
        number = int(match.group())
        return number_to_words(number)

    # Find the numbers and change with words.
    result = re.sub(r'\b\d+\b', replace, text)

    return result

In [ ]:
# Function to clean up text using the replacement pairs
def cleanup_text(text):
    for src, dst in replacements:
        text = text.replace(src, dst)
    return text

In [ ]:
converted_text = replace_numbers_with_words(text)
cleaned_text = cleanup_text(converted_text)
final_text = normalize_text(cleaned_text)
final_text

'keamn achho, amei tomaakea vaalbaasei!'

In [ ]:
# Function to clean up text using the replacement pairs
def cleanup_text(text):
    """
    This function takes a string as input and applies a series of replacements
    defined in the 'replacements' variable. It iterates through the
    replacement pairs and replaces all occurrences of the source string with
    the destination string within the input text.

    Args:
        text (str): The input text string to be cleaned up.

    Returns:
        str: The cleaned-up text string with the replacements applied.
    """
    for src, dst in replacements:
        # Directly replace within the input 'text' string
        text = text.replace(src, dst)
    return text

In [ ]:
inputs = processor(text=final_text, return_tensors="pt")

In [ ]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [ ]:
from IPython.display import Audio
import soundfile as sf

Audio(speech.numpy(), rate=16000)
# Save the audio to a file (e.g., 'output.wav')
sf.write('output.wav', speech.numpy(), 16000)